In [1]:
from config import config
from feature_generation import get_all_atms_feature_set
from preprocessing import get_input_sets, scaler_fit_transform, scaler_transform, scaler_inverse_transform
from tabTransformer import TabTransformer
from misc import nmae_error, load_pickle

import optuna
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Load Data

In [2]:
load_config = config['load_config']
clusters = load_config['clusters']

df = pd.read_csv(load_config['path'])
all_atms_feature_set = get_all_atms_feature_set(df, first_n = load_config['n_atms'])
all_atms_feature_set.sort_index(inplace = True)

# Reading Pickles
for cluster_feature in clusters:
    d = load_pickle(clusters[cluster_feature]['path'])
    all_atms_feature_set[cluster_feature] = all_atms_feature_set['AtmId'].map(d)

## Setting Features

In [3]:
feature_config  = config['feature_config']

categorical_features = [cat for cat in
    all_atms_feature_set.select_dtypes(include=feature_config['categorical_column_types'])
    if cat not in feature_config['excluded_categorical']]
continuous_features = [cat for cat in
    all_atms_feature_set.select_dtypes(include=feature_config['continuous_column_types'])
    if cat not in feature_config['excluded_continuous']]

groups = [continuous_features]
groups.extend(categorical_features)

## Aranging train/test Data

In [4]:
X = all_atms_feature_set[continuous_features + categorical_features]
y = all_atms_feature_set[feature_config['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)

# MinMaxTransform
X_train, y_train, scaler_X, scaler_y = scaler_fit_transform(X_train, y_train, continuous_features)
X_test, y_test = scaler_transform(X_test, y_test, scaler_X, scaler_y, continuous_features)

X_train = get_input_sets(X_train, groups)
X_test  = get_input_sets(X_test, groups)

c:\Users\cahid\Documents\GitHub\atm-demand-forecasting\preprocessing.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numerical_features] = scaler_X.transform(X[numerical_features])
C:\Users\cahid\anaconda3\envs\tensor\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
c:\Users\cahid\Documents\GitHub\atm-demand-forecasting\preprocessing.py:29: SettingWithCopyWarning: 
A value is trying to be set on a 

## Optuna

In [5]:
def objective(trial):

    param = {
        'dim': trial.suggest_categorical('dim', [32,64]),
        'depth': trial.suggest_categorical('depth', [1,2,3,6,12]),
        'heads': trial.suggest_categorical('heads', [2,4,8]),
        'attn_dropout': trial.suggest_categorical('attn_dropout', [0,0.1,0.2,0.3,0.4,0.5]),
        'ff_dropout': trial.suggest_categorical('ff_dropout', [0,0.1,0.2,0.3,0.4,0.5])
    }

    mlp_activation = trial.suggest_categorical('mlp_activation', ['relu', 'selu'])
    mlp_hidden = [
        (trial.suggest_categorical('mlp_1_dim', [256, 128, 64]), mlp_activation),
        (trial.suggest_categorical('mlp_2_dim', [64, 32, 16]), mlp_activation),
    ]

    tabTransformer = TabTransformer(
        categories = [len(all_atms_feature_set[cat].unique())
        if cat not in clusters.keys() else
        clusters[cat]['n_clusters']
        for cat in categorical_features],
        num_continuous = len(continuous_features),
        dim_out=1,
        mlp_hidden=mlp_hidden,
        **param
    )

    training_config = config['training_config']

    tabTransformer.compile(
        optimizer = tf.optimizers.Adam(learning_rate = trial.suggest_loguniform('learning_rate', 0.005, 0.5)),
        loss = training_config['loss']
    )

    tabTransformer.fit(
        X_train,
        y_train,
        epochs = 5,
        batch_size = training_config['batch_size'],
        verbose = False)

    error = nmae_error(scaler_inverse_transform(y_test, scaler_y), scaler_y.inverse_transform(tabTransformer.predict(X_test)))
    print(error)

    return error

In [14]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)


print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)

[I 2021-08-10 09:53:43,031] A new study created in memory with name: no-name-9b9e42e2-968d-4da0-88e3-c69335c8f801
[I 2021-08-10 09:54:05,440] Trial 0 finished with value: 0.49498965698860153 and parameters: {'dim': 32, 'depth': 1, 'heads': 8, 'attn_dropout': 0.3, 'ff_dropout': 0.5, 'mlp_activation': 'selu', 'mlp_1_dim': 128, 'mlp_2_dim': 16, 'learning_rate': 0.01316667365976463}. Best is trial 0 with value: 0.49498965698860153.


0.49498965698860153


[I 2021-08-10 09:54:23,839] Trial 1 finished with value: 0.4793972493396823 and parameters: {'dim': 64, 'depth': 2, 'heads': 2, 'attn_dropout': 0, 'ff_dropout': 0.1, 'mlp_activation': 'relu', 'mlp_1_dim': 128, 'mlp_2_dim': 32, 'learning_rate': 0.017904088592949254}. Best is trial 1 with value: 0.4793972493396823.


0.4793972493396823


[I 2021-08-10 09:54:36,948] Trial 2 finished with value: 0.3310462785938368 and parameters: {'dim': 64, 'depth': 1, 'heads': 4, 'attn_dropout': 0.5, 'ff_dropout': 0.5, 'mlp_activation': 'selu', 'mlp_1_dim': 128, 'mlp_2_dim': 16, 'learning_rate': 0.006973916021973657}. Best is trial 2 with value: 0.3310462785938368.


...

0.4514522285186496


[I 2021-08-10 10:07:15,347] Trial 18 finished with value: 0.4791465482005055 and parameters: {'dim': 64, 'depth': 6, 'heads': 4, 'attn_dropout': 0.5, 'ff_dropout': 0.5, 'mlp_activation': 'selu', 'mlp_1_dim': 64, 'mlp_2_dim': 16, 'learning_rate': 0.033778457868703}. Best is trial 13 with value: 0.32449295287244573.


0.4791465482005055


[I 2021-08-10 10:08:00,747] Trial 19 finished with value: 10.574422360584437 and parameters: {'dim': 64, 'depth': 2, 'heads': 8, 'attn_dropout': 0.5, 'ff_dropout': 0.3, 'mlp_activation': 'selu', 'mlp_1_dim': 256, 'mlp_2_dim': 16, 'learning_rate': 0.06602838449972318}. Best is trial 13 with value: 0.32449295287244573.


10.574422360584437
Number of finished trials: 20
Best trial: {'dim': 64, 'depth': 6, 'heads': 4, 'attn_dropout': 0.5, 'ff_dropout': 0.3, 'mlp_activation': 'selu', 'mlp_1_dim': 64, 'mlp_2_dim': 16, 'learning_rate': 0.009197764665216707}
Best value: 0.32449295287244573


In [15]:
import json
params = study.best_trial.params
with open('hyperparameters.txt', 'w') as outfile:
    json.dump(params, outfile)

In [16]:
params

{'dim': 64,
 'depth': 6,
 'heads': 4,
 'attn_dropout': 0.5,
 'ff_dropout': 0.3,
 'mlp_activation': 'selu',
 'mlp_1_dim': 64,
 'mlp_2_dim': 16,
 'learning_rate': 0.009197764665216707}